<a href="https://colab.research.google.com/github/KotzMarsa/big-data-project/blob/master/aac_intakes_and_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import bibliotek i danych

In [0]:
import pandas
import numpy as np

In [0]:
intakes_filename = 'https://raw.githubusercontent.com/KotzMarsa/big-data-project/master/Austin_Animal_Center_Intakes.csv'
outcomes_filename = 'https://raw.githubusercontent.com/KotzMarsa/big-data-project/master/Austin_Animal_Center_Outcomes.csv'

In [0]:
intakes = pandas.read_csv(intakes_filename)
outcomes = pandas.read_csv(outcomes_filename)

In [60]:
print(intakes.columns)
print(outcomes.columns)

Index(['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Found Location',
       'Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake',
       'Age upon Intake', 'Breed', 'Color'],
      dtype='object')
Index(['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Date of Birth',
       'Outcome Type', 'Outcome Subtype', 'Animal Type', 'Sex upon Outcome',
       'Age upon Outcome', 'Breed', 'Color'],
      dtype='object')


# Przekształcenie danych

Zmiana nazw i typu kolumn zawierających daty

In [0]:
intakes.rename(columns={'DateTime':'IntakeDateTime'}, inplace=True)
outcomes.rename(columns={'DateTime':'OutcomeDateTime'}, inplace=True)
intakes = intakes.drop(['MonthYear'], axis=1)
outcomes = outcomes.drop(['MonthYear'], axis=1)

In [0]:
intakes['IntakeDateTime'] = pandas.to_datetime(intakes['IntakeDateTime'])
outcomes['OutcomeDateTime'] = pandas.to_datetime(outcomes['OutcomeDateTime'])

Usunięcie duplikatów i problematycznych danych (błędnie wprowadzone daty)

In [0]:
intakes.drop_duplicates(keep='first', inplace=True)
outcomes.drop_duplicates(keep='first', inplace=True)
outcomes = outcomes[outcomes['OutcomeDateTime'] != '2014-07-06 15:58:00']
outcomes = outcomes[outcomes['OutcomeDateTime'] != '2017-02-01 10:15:00']
intakes = intakes[intakes['IntakeDateTime'] != '2019-04-26 13:26:00']
outcomes = outcomes[outcomes['OutcomeDateTime'] != '2019-01-12 15:17:00']
intakes = intakes[intakes['IntakeDateTime'] != '2018-02-08 16:12:00']
outcomes = outcomes[outcomes['OutcomeDateTime'] != '2016-07-24 19:25:00']

Połączenie obu zbiorów

In [0]:
intakes = intakes.sort_values(by=['IntakeDateTime'])
outcomes = outcomes.sort_values(by=['OutcomeDateTime'])

In [0]:
intakes['Counter'] = intakes.groupby('Animal ID').cumcount()
outcomes['Counter'] = outcomes.groupby('Animal ID').cumcount()
intakes = intakes.sort_values(by=['IntakeDateTime'])
outcomes = outcomes.sort_values(by=['OutcomeDateTime'])

In [0]:
intakes['new id'] = intakes['Animal ID'] + intakes['Counter'].map(str)
outcomes['new id'] = outcomes['Animal ID'] + outcomes['Counter'].map(str)
intakes = intakes.sort_values(by=['IntakeDateTime'])
outcomes = outcomes.sort_values(by=['OutcomeDateTime'])

In [0]:
in_and_out = intakes.merge(outcomes, how='left', on='new id')

In [0]:
to_drop = (in_and_out[in_and_out['OutcomeDateTime'] <= in_and_out['IntakeDateTime']])
to_drop = to_drop[to_drop['Counter_x'] == 0]

In [0]:
cond = outcomes['new id'].isin(to_drop['new id']) == True
outcomes.drop(outcomes[cond].index, inplace = True)

In [0]:
outcomes = outcomes.drop(['Counter'], axis=1)
outcomes = outcomes.drop(['new id'], axis=1)
outcomes = outcomes.sort_values(by=['OutcomeDateTime'])
outcomes['cc'] = outcomes.groupby('Animal ID').cumcount()
outcomes = outcomes.sort_values(by=['OutcomeDateTime'])
outcomes['new id'] = outcomes['Animal ID'] + outcomes['cc'].map(str)

In [0]:
in_and_out = intakes.merge(outcomes, how='left').drop('new id', 1)

In [0]:
in_and_out['Date of Birth'] = pandas.to_datetime(in_and_out['Date of Birth'])

In [0]:
in_and_out = in_and_out.drop(['cc'], axis=1)
in_and_out['Counter'] = in_and_out['Counter'] + 1

Wprowadzenie dodatkowych kolumn:


*   Time in AAC
*   Is in AAC



In [0]:
in_and_out['Time in AAC'] = abs(in_and_out['OutcomeDateTime'] - in_and_out['IntakeDateTime'])

In [0]:
in_and_out['Is in AAC'] = ~pandas.notna(in_and_out['OutcomeDateTime'])

In [0]:
to_drop = (in_and_out[in_and_out['Time in AAC'] < '00:00:00'])

Usunięcie symbolu * z imion (ponieważ brak informacji o tym, co miał oznaczać)

In [0]:
in_and_out['Name'] = in_and_out['Name'].str.replace('*', '')
in_and_out['Name'] = in_and_out['Name'].str.replace('Ă‰', 'É')

Dodanie nazw dni tygodnia

In [0]:
in_and_out['Day of intake'] = in_and_out['IntakeDateTime'].dt.day_name()
in_and_out['Day of outcome'] = in_and_out['OutcomeDateTime'].dt.day_name()

Zmiana kolejności kolumn

In [0]:
in_and_out = in_and_out.reindex(columns=['Animal ID', 
                                         'Animal Type', 'Name', 'Breed', 'Color', 'Date of Birth', 
                                         'IntakeDateTime', 'OutcomeDateTime', 'Day of intake', 'Day of outcome', 'Counter', 'Found Location',
                                         'Intake Type', 'Intake Condition', 'Sex upon Intake', 'Sex upon Outcome',
                                         'Age upon Intake', 'Age upon Outcome', 'Outcome Type', 'Outcome Subtype', 'Is in AAC', 'Time in AAC'])

Rozdzielenie kolumn zawierających datę i czas

In [0]:
in_and_out['Intake Date'] = pandas.to_datetime(in_and_out['IntakeDateTime']).dt.date
in_and_out['Intake Time'] = pandas.to_datetime(in_and_out['IntakeDateTime']).dt.time
in_and_out['Outcome Date'] = pandas.to_datetime(in_and_out['OutcomeDateTime']).dt.date
in_and_out['Outcome Time'] = pandas.to_datetime(in_and_out['OutcomeDateTime']).dt.time

Dodanie liczby dni spędzonych przez zwierzęta w AAC.

Kolumna *Days in AAC (not adopted)* jest obliczona tutaj dla wszystkich i będzie filtrowana dopiero w Power Bi.

In [0]:
in_and_out['Days in AAC (adopted)'] = in_and_out['Outcome Date'] - in_and_out['Intake Date']

In [0]:
today = pandas.to_datetime('2019-12-10')
in_and_out['Days in AAC (not adopted)'] = (today - pandas.to_datetime(in_and_out['Intake Date']))

# Gotowy zbiór danych i informacje o nim

In [83]:
in_and_out

,Animal ID,Animal Type,Name,Breed,Color,Date of Birth,IntakeDateTime,OutcomeDateTime,Day of intake,Day of outcome,Counter,Found Location,Intake Type,Intake Condition,Sex upon Intake,Sex upon Outcome,Age upon Intake,Age upon Outcome,Outcome Type,Outcome Subtype,Is in AAC,Time in AAC,Intake Date,Intake Time,Outcome Date,Outcome Time,Days in AAC (adopted),Days in AAC (not adopted)
0,A521520,Dog,Nina,Border Terrier/Border Collie,White/Tan,2006-09-07,2013-10-01 07:51:00,2013-10-01 15:39:00,Tuesday,Tuesday,1,Norht Ec in Austin (TX),Stray,Normal,Spayed Female,Spayed Female,7 years,7 years,Return to Owner,NaN,False,07:48:00,2013-10-01,07:51:00,2013-10-01,15:39:00,0 days,2261 days
1,A664235,Cat,NaN,Domestic Shorthair Mix,Orange/White,2013-09-24,2013-10-01 08:33:00,2013-10-01 10:39:00,Tuesday,Tuesday,1,Abia in Austin (TX),Stray,Normal,Unknown,Unknown,1 week,1 week,Transfer,Partner,False,02:06:00,2013-10-01,08:33:00,2013-10-01,10:39:00,0 days,2261 days
2,A664236,Cat,NaN,Domestic Shorthair Mix,Orange/White,2013-09-24,2013-10-01 08:33:00,2013-10-01 10:44:00,Tuesday,Tuesday,1,Abia in Austin (TX),Stray,Normal,Unknown,Unknown,1 week,1 week,Transfer,Partner,False,02:11:00,2013-10-01,08:33:00,2013-10-01,10:44:00,0 days,2261 days
3,A664237,Cat,NaN,Domestic Shorthair Mix,Orange/White,2013-09-24,2013-10-01 08:33:00,2013-10-01 10:44:00,Tuesday,Tuesday,1,Abia in Austin (TX),Stray,Normal,Unknown,Unknown,1 week,1 week,Transfer,Partner,False,02:11:00,2013-10-01,08:33:00,2013-10-01,10:44:00,0 days,2261 days
4,A664233,Dog,Stevie,Pit Bull Mix,Blue/White,2010-09-30,2013-10-01 08:53:00,2013-10-01 15:33:00,Tuesday,Tuesday,1,7405 Springtime in Austin (TX),Stray,Injured,Intact Female,Intact Female,3 years,3 years,Euthanasia,Suffering,False,06:40:00,2013-10-01,08:53:00,2013-10-01,15:33:00,0 days,2261 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112600,A810187,Dog,Canelo,American Pit Bull Terrier/Boxer,Tan/White,NaT,2019-12-08 16:38:00,NaT,Sunday,NaN,1,Austin (TX),Owner Surrender,Normal,Neutered Male,NaN,9 months,NaN,NaN,NaN,True,NaT,2019-12-08,16:38:00,NaT,NaT,NaT,2 days
112601,A810189,Dog,Daisy,Treeing Cur Mix,Black/White,NaT,2019-12-08 17:31:00,NaT,Sunday,NaN,1,Sweet Leaf And Harcourt House in Pflugerville ...,Stray,Normal,Intact Female,NaN,2 years,NaN,NaN,NaN,True,NaT,2019-12-08,17:31:00,NaT,NaT,NaT,2 days
112602,A810190,Other,NaN,Raccoon,Gray/Black,2017-12-08,2019-12-08 18:21:00,2019-12-08 18:54:00,Sunday,Sunday,1,5114 Eilers Avenue in Austin (TX),Wildlife,Sick,Unknown,Unknown,2 years,2 years,Euthanasia,Rabies Risk,False,00:33:00,2019-12-08,18:21:00,2019-12-08,18:54:00,0 days,2 days
112603,A790482,Dog,Blue,Pit Bull Mix,Gray/White,NaT,2019-12-08 21:44:00,NaT,Sunday,NaN,4,5902 Broken Spoke Trail in Austin (TX),Stray,Normal,Neutered Male,NaN,2 years,NaN,NaN,NaN,True,NaT,2019-12-08,21:44:00,NaT,NaT,NaT,2 days


In [84]:
pandas.isnull(in_and_out).sum()

Animal ID                        0
Animal Type                      0
Name                         35639
Breed                            0
Color                            0
Date of Birth                 1487
IntakeDateTime                   0
OutcomeDateTime               1487
Day of intake                    0
Day of outcome                1487
Counter                          0
Found Location                   0
Intake Type                      0
Intake Condition                 0
Sex upon Intake                  1
Sex upon Outcome              1491
Age upon Intake                  0
Age upon Outcome              1508
Outcome Type                  1494
Outcome Subtype              62399
Is in AAC                        0
Time in AAC                   1487
Intake Date                      0
Intake Time                      0
Outcome Date                  1487
Outcome Time                  1487
Days in AAC (adopted)         1487
Days in AAC (not adopted)        0
dtype: int64

In [85]:
in_and_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112605 entries, 0 to 112604
Data columns (total 28 columns):
Animal ID                    112605 non-null object
Animal Type                  112605 non-null object
Name                         76966 non-null object
Breed                        112605 non-null object
Color                        112605 non-null object
Date of Birth                111118 non-null datetime64[ns]
IntakeDateTime               112605 non-null datetime64[ns]
OutcomeDateTime              111118 non-null datetime64[ns]
Day of intake                112605 non-null object
Day of outcome               111118 non-null object
Counter                      112605 non-null int64
Found Location               112605 non-null object
Intake Type                  112605 non-null object
Intake Condition             112605 non-null object
Sex upon Intake              112604 non-null object
Sex upon Outcome             111114 non-null object
Age upon Intake              112605

# Eksport do pliku csv

In [0]:
from google.colab import files
in_and_out.to_csv('filename.csv') 
files.download('filename.csv')